In [97]:
"""
Creates a network visualization of transactions between refund receivers
"""
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime
import requests


In [98]:
def get_wallet_transactions(wallet_address: str, after_timestamp: int) -> list:
    """Fetch transactions for a specific wallet after given timestamp"""
    url = f"https://tonapi.io/v2/blockchain/accounts/{wallet_address}/transactions"
    params = {"limit": 100}  # Adjust limit as needed

    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=30)
        response.raise_for_status()
        return [tx for tx in response.json().get("transactions", []) 
                if tx.get("created_at", 0) > after_timestamp]
    except Exception as e:
        print(f"Error fetching transactions for {wallet_address}: {e}")
        return []


In [99]:
def create_transaction_network(refund_transactions):
    """Create and visualize transaction network"""
    G = nx.DiGraph()

    # Add nodes and edges
    for wallet in refund_receivers:
        G.add_node(wallet[:8] + "...", size=1000)  # Truncate address for readability

        # Get transactions from this wallet
        transactions = get_wallet_transactions(wallet, 0)  # Add proper timestamp

        for tx in transactions:
            for msg in tx.get("out_msgs", []):
                dest = msg.get("destination", {}).get("address")
                if dest:
                    dest_short = dest[:8] + "..."
                    G.add_node(dest_short, size=500)
                    G.add_edge(wallet[:8] + "...", dest_short, 
                             weight=msg.get("value", 0) / 1e9)  # Convert to TON
    return G


In [100]:
def visualize_network(G):
    """Create and display network visualization"""
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, k=1, iterations=50)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_color='lightblue',
                          node_size=[G.nodes[node].get('size', 300) for node in G.nodes()],
                          alpha=0.6)

    # Draw edges with varying width based on transaction value
    edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, 
                          width=[max(0.5, min(2, w)) for w in edge_weights],
                          alpha=0.4,
                          edge_color='gray')

    # Add labels
    nx.draw_networkx_labels(G, pos, font_size=8)

    plt.title("Transaction Network from Refund Recipients")
    plt.axis('off')
    plt.show()


In [101]:
# Let's check the API limits and response details
def check_api_limits():
    test_wallet = next(iter(refund_wallets))
    url = f"https://tonapi.io/v2/blockchain/accounts/{test_wallet}/transactions"
    params = {"limit": 100}

    print("Testing API with wallet:", test_wallet[:8] + "...")
    try:
        response = requests.get(url, headers=HEADERS, params=params)

        print("\nResponse Status:", response.status_code)
        print("\nResponse Headers:")
        for header, value in response.headers.items():
            print(f"{header}: {value}")

        if response.status_code == 429:
            print("\nRate Limit Response Body:")
            print(response.text)

        print("\nCurrent Headers being sent:")
        print("HEADERS =", HEADERS)

    except Exception as e:
        print(f"Error checking API limits: {e}")

check_api_limits()


Testing API with wallet: 0:325501...

Response Status: 200

Response Headers:
Date: Thu, 19 Jun 2025 10:08:48 GMT
Content-Type: application/json; charset=utf-8
Transfer-Encoding: chunked
Connection: keep-alive
access-control-allow-origin: *
access-control-allow-credentials: true
access-control-allow-methods: GET, POST, OPTIONS
access-control-allow-headers: DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Authorization,X-Tonapi-Client
Content-Encoding: gzip
cf-cache-status: DYNAMIC
vary: accept-encoding
Server: cloudflare
CF-RAY: 95223a2e5964d625-LCA
alt-svc: h3=":443"; ma=86400

Current Headers being sent:
HEADERS = {'accept': 'application/json'}


In [102]:
# First, let's collect all transactions between refund wallets
refund_wallets = set(refund_receivers)  # Convert to set for faster lookups
connections = []

for i, source_wallet in enumerate(refund_wallets):
    print(f"Processing wallet {i + 1}/{len(refund_wallets)}: {source_wallet[:8]}...")

    try:
        transactions = get_wallet_transactions(source_wallet, 0)

        for tx in transactions:
            for msg in tx.get("out_msgs", []):
                dest = msg.get("destination", {}).get("address")
                if dest in refund_wallets:  # Only track if destination is also a refund wallet
                    connections.append({
                        'from': source_wallet,
                        'to': dest,
                        'amount': msg.get("value", 0) / 1e9,  # Convert to TON
                        'timestamp': tx.get("created_at", 0)
                    })

    except Exception as e:
        print(f"Error processing {source_wallet[:8]}...: {str(e)}")
        continue

print(f"\nProcessed all {len(refund_wallets)} wallets. Found {len(connections)} connections.")


Processing wallet 1/14: 0:325501...
Processing wallet 2/14: 0:3f6076...
Processing wallet 3/14: 0:d00ac9...
Processing wallet 4/14: 0:b1ccf9...
Processing wallet 5/14: 0:22d4e1...
Processing wallet 6/14: 0:9d894e...
Processing wallet 7/14: 0:daa21f...
Processing wallet 8/14: 0:25dbba...
Processing wallet 9/14: 0:35d019...
Processing wallet 10/14: 0:9aa3d6...
Processing wallet 11/14: 0:b868d5...
Error fetching transactions for 0:b868d5092c9344d413df0095e7a0db91e407500b7b0e3b7ddcb9ae385a7f9e83: 429 Client Error: Too Many Requests for url: https://tonapi.io/v2/blockchain/accounts/0:b868d5092c9344d413df0095e7a0db91e407500b7b0e3b7ddcb9ae385a7f9e83/transactions?limit=100
Processing wallet 12/14: 0:5b892b...
Error fetching transactions for 0:5b892bed4f3a282ec9e88b27ed528f00fb3802268c1fcc408b8c9d09f901f86b: 429 Client Error: Too Many Requests for url: https://tonapi.io/v2/blockchain/accounts/0:5b892bed4f3a282ec9e88b27ed528f00fb3802268c1fcc408b8c9d09f901f86b/transactions?limit=100
Processing wa

In [103]:
# Print the connections in a readable format
if connections:
    print("Found the following transactions between refund wallets:")
    for conn in connections:
        print(f"From: {conn['from'][:8]}...")
        print(f"To: {conn['to'][:8]}...")
        print(f"Amount: {conn['amount']:.2f} TON")
        print(f"Time: {datetime.fromtimestamp(conn['timestamp']).strftime('%Y-%m-%d %H:%M:%S')}")
        print("-" * 50)
else:
    print("No transactions found between refund wallets!")


No transactions found between refund wallets!


In [104]:
# Create a visualization of only the connected refund wallets
G = nx.DiGraph()

# Add edges only for wallets that have transactions between them
for conn in connections:
    from_short = conn['from'][:8] + "..."
    to_short = conn['to'][:8] + "..."
    G.add_edge(from_short, to_short, weight=conn['amount'])

if G.number_of_edges() > 0:
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, k=1, iterations=50)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_color='lightblue',
                          node_size=1000,
                          alpha=0.6)

    # Draw edges with varying width based on transaction value
    edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, 
                          width=[max(0.5, min(2, w)) for w in edge_weights],
                          alpha=0.4,
                          edge_color='red',
                          arrows=True,
                          arrowsize=20)

    # Add labels
    nx.draw_networkx_labels(G, pos, font_size=8)

    plt.title("Transactions Network Between Refund Recipients")
    plt.axis('off')
    plt.show()
else:
    print("No connections to visualize!")


No connections to visualize!
